In [28]:
# You need to install the OpenSkyAPI library
# Get it here: https://github.com/openskynetwork/opensky-api

In [29]:
# Install it
#!pip install -e ./opensky-api/python
#!pip install geocoder
#conda install basemap

In [30]:


from opensky_api import OpenSkyApi
import geocoder
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests

In [31]:

import os
import conda

#Fix Error on Basemap Environment Variable

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib


from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  
import warnings
import matplotlib.cbook



In [32]:
# API request
#latmin latmax lngmin lngmax
# generous rectangle around Germany (Novatim Server)

api = OpenSkyApi()
states = api.get_states(bbox=(47.2701114,55.099161,5.8663153,15.0419319))


In [33]:
# Delete your pre-existing dataframe
try:
    del df
except: Exception

        

# Create your new Dataframe
df = pd.DataFrame(columns=['longitude', 'latitude', 'velocity', 'callsign', 'origin_country', 'on_ground', 'squawk', 'vertical_rate', 'icao24'])

In [34]:
# Fill your dataframe - again, lazy solution, 'cause the read_json didn't work


for s in states.states:
    df.loc[s] = [ s.longitude, s.latitude, s.velocity, s.callsign, s.origin_country, s.on_ground, s.squawk, s.vertical_rate, s.icao24 ]
df.reset_index(inplace=True)

In [43]:
#print('All Planes in the Sky over Germany')
#df

In [44]:
url = "https://ae.roplan.es/api/hex-type.php?hex="
headers = {
    'User-Agent': 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us'
}

In [45]:
# Function to get the aircraft type from ae.roplan.es
# I know, you wouldn't need it here but I used a different website with BS4 etc
def get_type(callsign):
    r = requests.get(url+callsign, headers=headers)
    return (r.text)

In [46]:
# Test it
# get_type("49d027")

In [47]:
# Add aircraft type to dataframe
for index,row in df.iterrows():
    df.loc[index,'type'] = get_type(row['icao24'])

In [48]:
# Filter dataframe by aircraft type
result = df[df['type'].astype(str).str.contains('Boeing 737')]

In [ ]:
# Get your own Lat/Long position
g = geocoder.ip('me')
#print(g.latlng)

fig = plt.figure(figsize=(8, 10))

m = Basemap(width=770000, height=909000,
            resolution=None, projection='lcc',
            lon_0=11, lat_0=51.5)


m.shadedrelief()


for index, row in result.iterrows():
    print(row['longitude'], row['latitude'], row['callsign'])
    x, y = m(row['longitude'], row['latitude'])
    plt.plot(x-4, y, 'o', markersize=12)
    plt.text(x, y, row['callsign'], fontsize=12);
    
    x, y = m(9.99,53.5)
    plt.plot(x-1, y, '+', markersize=12)
    plt.text(x, y, 'Hamburg', fontsize=12);

    x, y = m(9.73828,52.37443)
    plt.plot(x-1, y, '+', markersize=12)
    plt.text(x, y, 'Hannover', fontsize=12);
    
    x, y = m(11.55915,48.14128)
    plt.plot(x-1, y, '+', markersize=12)
    plt.text(x, y, 'München', fontsize=12);
    
    x, y = m(13.29386,52.60021)
    plt.plot(x-1, y, '+', markersize=12)
    plt.text(x, y, 'Berlin', fontsize=12);
    
   
    
    
    
